In [61]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [7]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We will be working with the same medical data used in the `pw` miniproject, but will be leveraging the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [169]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
--2019-06-09 07:27:21--  http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.216.237.131
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.216.237.131|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19011654 (18M) [text/csv]
Saving to: ‘./dw-data/201701scripts_sample.csv.gz’

201701scripts_sampl 100%[===================>]  18.13M  40.6MB/s    in 0.4s    

2019-06-09 07:27:21 (40.6 MB/s) - ‘./dw-data/201701scripts_sample.csv.gz’ saved [19011654/19011654]

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

--2019-06-09 07:27:22--  http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.216.237.131
Connecting to dataincubator-wqu.s3.amazona

## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [3]:
import pandas as pd
import numpy as np
import gzip

In [25]:
# load the 2017 data
with gzip.open('./dw-data/201701scripts_sample.csv.gz', 'rb') as f:
    scripts = pd.read_csv(f)

In [18]:
with gzip.open('./dw-data/practices.csv.gz', 'rb') as f:
    practices = pd.read_csv(f)
practices.head()
col_names=[ 'practice', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices.columns = col_names

In [188]:
with gzip.open('./dw-data/chem.csv.gz', 'rb') as f:
    chem = pd.read_csv(f)

Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [8]:
def describe(key):
    total = 0
    avg = 0
    s = 0
    q25 = 0
    med = 0
    q75 = 0
    total = scripts[key].sum()
    avg = scripts[key].mean()
    s = scripts[key].std()
    q25 = scripts[key].quantile(0.25)
    med = scripts[key].quantile(0.5)
    q75 = scripts[key].quantile(0.75)
    return (total, avg, s, q25, med, q75)

summary_stats = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [9]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [10]:
bnf_names = scripts.groupby(['bnf_name'])
groups = {name[0]: name[1]['items'].sum() for name in bnf_names}
key_max = max(groups.keys(), key=(lambda k: groups[k]))
most_common_item = [(key_max, groups[key_max])]

In [11]:
grader.score.dw__most_common_item(most_common_item)

Your score:  1.0


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [15]:
with gzip.open('./dw-data/practices.csv.gz', 'rb') as f:
    practices = pd.read_csv(f)
practices.head()
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices.columns = col_names
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
1,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
2,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
3,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ
4,A81006,TENNANT STREET MEDICAL PRACTICE,TENNANT ST MED PRACT,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AT


In [13]:
practices = practices.sort_values(by=["post_code"]).groupby(["code"]).first().reset_index()
joined = pd.merge(scripts, practices, how="inner",left_on="practice",right_on="code")
postal_totals = joined.groupby(['post_code']).sum().reset_index().rename(columns={"items":"items_y"})
total_by_bnf = joined.groupby(['bnf_name','post_code']).sum().reset_index()
total_by_bnf = total_by_bnf.sort_values(by=["bnf_name","items"]).groupby(["post_code","items"]).first().reset_index().groupby(["post_code","items"]).filter(lambda x:len(x)>1).reset_index()
print(total_by_bnf)

Empty DataFrame
Columns: [index, post_code, items, bnf_name, nic, act_cost, quantity]
Index: []


In [ ]:
scripts = scripts.rename(columns={'practice':'code'})
practices.sort_values(by=['post_code'], inplace = True)
practices.drop_duplicates(subset='code', keep = 'first') 
joined = pd.merge(scripts, practices, how = 'inner', on = 'code')
total_by_bnf = joined.groupby(['post_code','bnf_name'], as_index=False)['items'].sum()
postal_totals = joined.groupby(['post_code'], as_index=False)['items'].sum()
items_by_bnf_post = pd.merge(total_by_bnf, postal_totals, how = 'inner', on = 'post_code')
items_by_bnf_post.sort_values(by=['items_x'], ascending = False, inplace = True)
items_by_bnf_post = items_by_bnf_post.drop_duplicates(subset='post_code', keep = 'first')
items_by_bnf_post['rate'] = items_by_bnf_post['items_x'] / items_by_bnf_post['items_y']
items_by_bnf_post.sort_values(by=['post_code'], ascending = True, inplace = True )
items_by_bnf_post.drop(['items_x', 'items_y'], axis = 1, inplace = True )
items_by_region = list(map(lambda x,y,z:(x,y,z),items_by_bnf_post["post_code"],items_by_bnf_post["bnf_name"],items_by_bnf_post["rate"]))[:100]

In [ ]:
grader.score.dw__items_by_region(items_by_region)

## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [ ]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [ ]:
pattern = "|".join(opioids)
o_chem = chem[chem["NAME"].str.contains(pattern, case=False, regex=True)]
opioid_codes = o_chem["CHEM SUB"].values
code_pattern = "|".join(opioid_codes)
scripts["is_opioid"] = scripts["bnf_code"].str.contains(code_pattern, case=False, regex=True)

Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [ ]:
opioids_per_practice = pd.DataFrame(script_content.groupby('practice')['opioid_content'].mean())

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [ ]:
scripts_by_practice = scripts.groupby("practice")
opioids_per_practice = scripts_by_practice["is_opioid"].mean()

relative_opioids_per_practice = opioids_per_practice - scripts["is_opioid"].mean()

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [ ]:
standard_error_per_practice = scripts["is_opioid"].std() / scripts_by_practice["practice"].count().apply(np.sqrt)
opioid_scores = pd.DataFrame(relative_opioids_per_practice / standard_error_per_practice).reset_index()
opioid_scores.columns = ["code","z_score"]

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [ ]:
joinedDf = pd.merge(practices[['code','name']],opioid_scores, how="inner")
countDf = scripts.groupby('practice')['practice'].count()
joinedDf = joinedDf.join(countDf, on='code', how='inner',lsuffix='_left',rsuffix='_right')
joinedDf.sort_values(by=["z_score"],inplace=True, ascending=False)
anomalies = list(map(lambda x,y,z :(x,y,z),joinedDf['name'],joinedDf['z_score'],joinedDf["practice"]))[:100]

In [ ]:
grader.score.dw__script_anomalies(anomalies)

## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [ ]:
scripts = pd.read_csv('./dw-data/201701scripts_sample.csv.gz')
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz')

scripts16_2 = scripts16[['bnf_name', 'items']]
scripts16_2.reset_index(inplace=True)
scripts16_2 = pd.DataFrame(scripts16_2.groupby('bnf_name').count())
scripts16_2.reset_index(inplace=True)

scripts_2=scripts[['bnf_name', 'items']]
scripts_2.reset_index(inplace=True)
scripts_2 = pd.DataFrame(scripts_2.groupby('bnf_name').count())
scripts_2.reset_index(inplace=True)

scripts16_4 = scripts16_2.merge(scripts_2, on='bnf_name')#, how="left"
scripts16_4.reset_index(inplace=True)
scripts16_4['items_y'].fillna(0, inplace=True)

scripts16_4['rate'] = (scripts16_4['items_y'] - scripts16_4['items_x'])/scripts16_4['items_x']
scripts16_4 = scripts16_4[scripts16_4['items_x'] >= 50]

growth_rate = scripts16_4[['bnf_name','rate', 'items_x', ]] \
                                    .sort_values(by=['rate'], ascending=False) \
                                    .reset_index(drop=True)

top_50 = growth_rate.head(50)
bottom_50 = growth_rate.tail(50)
script_growth = [tuple(i) for i in pd.concat([top_50, bottom_50]).values]

In [ ]:
script_growth = [("Butec_Transdermal Patch 5mcg\/hr", 3.4677419355, 62.0)] * 100

In [ ]:
grader.score.dw__script_growth(script_growth)

## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [160]:
scripts_bnf = scripts.groupby(['bnf_code'])['practice'].count()
scripts_bnf = scripts_bnf.reset_index()
scripts_bnf.columns = ['bnf_code','count']
prob = scripts_bnf['count']/len(scripts)
scripts_bnf = pd.concat([scripts_bnf, prob], axis=1)
scripts_bnf.columns = ['bnf_code','count','p']
p = 1/(scripts['bnf_code'].nunique())
scripts['rare'] = scripts_bnf["p"]/p < 0.1

In [48]:
scripts_bnf = scripts.groupby(['bnf_code'])['practice'].count()
scripts_bnf = scripts_bnf.reset_index()
scripts_bnf.columns = ['bnf_code','count']
prob = scripts_bnf['count']/len(scripts)
scripts_bnf = pd.concat([scripts_bnf, prob], axis=1)
scripts_bnf.columns = ['bnf_code','count','p']
p = 1/(scripts['bnf_code'].nunique())
scripts = pd.merge(scripts,scripts_bnf, how='outer')
scripts['rare'] = scripts["p"]/p < 0.1
act_cost = scripts.groupby('practice').sum().reset_index()
rare_act_cost = scripts[scripts['rare']==True].groupby('practice')['act_cost'].sum().reset_index()
rare_act_cost = rare_act_cost.rename(columns={'act_cost':'rare_act_cost'})
rare_prac_prop = pd.merge(act_cost, rare_act_cost, how="inner")
rare_prac_prop['rare_prac_prop'] = rare_prac_prop['rare_act_cost']/rare_prac_prop['act_cost']
rare_prac_prop = rare_prac_prop.fillna(0)
rare_cost_across_pracs_prop = sum(scripts[scripts['rare']== True]['act_cost'])/sum(scripts['act_cost'])
rare_prac_prop['relative_rare_cost_prop'] = rare_prac_prop['rare_prac_prop'] - rare_cost_across_pracs_prop
print(rare_cost_across_pracs_prop)
std = np.std(rare_prac_prop['relative_rare_cost_prop'])
print("Standard Error is : ", std)
rare_prac_prop['z_score'] = rare_prac_prop['relative_rare_cost_prop']/std
practices.sort_values(by='name')
practices.drop_duplicates(subset='practice',inplace=True)
result_df = pd.merge(rare_prac_prop, practices[['practice','name']], how="outer")
result_df = result_df.fillna(0)
result_df.sort_values(by='z_score',ascending=False,inplace=True)
rare_scripts = list(map(lambda x,y,z : (x,y,z),result_df['practice'],result_df['name'],result_df['z_score']))[:100]
grader.score.dw__rare_scripts(rare_scripts)

0.01596290136094875
Standard Error is :  0.06349230045394581
Your score:  0.9300000000000006



Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [79]:
result_df = scripts.groupby('practice').sum().reset_index()
rare_script = scripts[scripts['rare']==True]
grouped_rare = rare_script.groupby(['practice'])['act_cost'].sum().reset_index().rename(columns={'act_cost':'rare_cost'})
result_df = pd.merge(result_df,grouped_rare, how='outer')
result_df = result_df.fillna(0)
result_df['rare_cost_prop'] = result_df['rare_cost']/result_df['act_cost']
result_df = result_df.fillna(0)

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [82]:
#result_df['relative_rare_cost_prop'] = result_df['rare_cost_prop'] - (scripts[scripts['rare']== True]['act_cost'].sum()/scripts['act_cost'].sum())
#result_df = result_df.fillna(0)

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [85]:
standard_errors = result_df['relative_rare_cost_prop'].std()
result_df['z_score'] = result_df['relative_rare_cost_prop']/standard_errors

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [88]:
result_df = result_df.rename(columns={'practice':'code'})
result_df = pd.merge(practices[['code','name']],result_df,how="inner")
result_df.sort_values(by=['z_score'],inplace=True, ascending=False)
result_df.drop_duplicates(subset=['name'],inplace=True)
rare_scripts = list(map(lambda x,y,z :(x,y,z),result_df['code'],result_df['name'],result_df["z_score"]))[:100]
print(rare_scripts)

[('N85647', 'WALLASEY WIC (VCH)_WIC VCH', 20.440731700917993), ('N85639', 'GP OOH VCH', 17.30361424057564), ('N81013', 'HIGH STREET SURGERY', 10.105716983321152), ('N81029', 'SOUTH PARK SURGERY', 6.023208323971127), ('A81005', 'SPRINGWOOD SURGERY', -0.02559448832502911), ('P88624', 'BL MEDICAL PRACTICE', -0.02559448832502911), ('P87020', '2/ST ANDREWS MEDICAL PRACTICE', -0.02559448832502911), ('P87036', '4/LOWER BROUGHTON MEDICAL PRACTICE', -0.02559448832502911), ('P88007', 'CHEADLE HULME HEALTH CTR 2', -0.02559448832502911), ('P88009', 'WOODLEY VILLAGE SURGERY', -0.02559448832502911), ('P88019', 'ALVANLEY FAMILY PRACTICE', -0.02559448832502911), ('P88023', 'HEALD GREEN HEALTH CENTRE 2', -0.02559448832502911), ('P88025', 'HULME HALL MEDICAL GROUP', -0.02559448832502911), ('P88042', 'HEALD GREEN HEALTH CENTRE 1', -0.02559448832502911), ('P88625', 'ARCHWOOD MEDICAL PRACTICE', -0.02559448832502911), ('P87008', 'WALKDEN MEDICAL PRACTICE', -0.02559448832502911), ('P89008', 'BEDFORD HOUSE ME

In [222]:
scripts_bnf = scripts.groupby(['bnf_code'])['practice'].count()
scripts_bnf = scripts_bnf.reset_index()
scripts_bnf.columns = ['bnf_code','count']
prob = scripts_bnf['count']/sum(scripts_bnf['count'])
scripts_bnf = pd.concat([scripts_bnf, prob], axis=1)
scripts_bnf.columns = ['bnf_code','count','p']
p = 1/(scripts['bnf_code'].nunique())
scripts['rare'] = scripts_bnf["p"]/p < 0.1

#calculate total act cost for all scripts for each practice. 
actcost = pd.DataFrame(scripts.groupby('practice')['act_cost'].sum())
actcost.columns = ['act_cost_tot']
actcost = actcost.reset_index()  

#calculate total act cost for all rare scripts.
rare = scripts.loc[scripts['rare'] == True]
rare_actcost = pd.DataFrame(rare.groupby('practice')['act_cost'].sum().reset_index())

#get the proportion of cost related to rare treatments for each practice
actcost = pd.merge(actcost, rare_actcost, how = 'outer', on = 'practice')
actcost['rare_cost_prop'] = actcost['act_cost']/actcost['act_cost_tot']
#actcost = actcost.fillna(0)
#find proportion of cost of rare treatments across all practices
cost_allrare = scripts.loc[scripts['rare']== True]['act_cost'].sum()/scripts['act_cost'].sum()

#calculate the std() and the zscore of the proportions for each practice
actcost['relative_rare_cost_prop'] = actcost['rare_cost_prop'] - cost_allrare
standard_errors = actcost['relative_rare_cost_prop'].std()
print(standard_errors)
#actcost=actcost.fillna(0)
actcost['z_score'] = actcost['relative_rare_cost_prop']/standard_errors

#merge the actcost with the practices df to have the names
practices_sort = practices.sort_values('name')
practices_sort.drop_duplicates(subset='practice',inplace=True)
actcost = pd.merge(actcost[['practice','z_score']], practices_sort[['practice', 'name']], how = 'inner')
#sort value by z-scores
actcost.sort_values(by = 'z_score', ascending = False, inplace = True)
#remove name duplicated
actcost.drop_duplicates(subset = 'name', inplace = True)
actcost.reset_index(drop = True, inplace = True)
#compute the rare scores
actcost.fillna(0)
rare_scores = [(x[0], x[2], x[1]) for i, x in actcost.iterrows() if i < 100]
rare_scripts = rare_scores
print(rare_scripts)

0.22673893495307695
[('N85647', 'WALLASEY WIC (VCH)_WIC VCH', 3.104444249888106), ('N85639', 'GP OOH VCH', 2.6279932889596376), ('N81013', 'HIGH STREET SURGERY', 1.5348097826879152), ('N81029', 'SOUTH PARK SURGERY', 0.9147771577272149), ('A81005', 'SPRINGWOOD SURGERY', nan), ('A81007', 'BANKHOUSE SURGERY', nan), ('A81011', 'CHADWICK PRACTICE', nan), ('A81012', 'WESTBOURNE MEDICAL CENTRE', nan), ('A81017', 'WOODBRIDGE PRACTICE', nan), ('A81018', 'BENTLEY MEDICAL PRACTICE', nan), ('A81021', 'NORMANBY MEDICAL CENTRE', nan), ('A81023', 'THE ENDEAVOUR PRACTICE', nan), ('A81029', 'PROSPECT SURGERY', nan), ('A81031', 'HAVELOCK GRANGE PRACTICE', nan), ('A81032', 'THE GARTH', nan), ('A81033', 'OAKFIELD MEDICAL PRACTICE', nan), ('A81034', 'THORNABY & BARWICK MEDICAL GROUP', nan), ('A81037', 'THE ERIMUS PRACTICE', nan), ('A81038', 'HIRSEL MEDICAL CENTRE', nan), ('A81040', 'MARSH HOUSE MEDICAL PRACTICE', nan), ('A81049', 'KINGS MEDICAL CENTRE', nan), ('A81052', 'THE GREEN HOUSE SURGERY', nan), ('A

In [223]:
grader.score.dw__rare_scripts(rare_scripts)

Your score:  0.09999999999999999


#### *Copyright &copy; 2019 The Data Incubator.  All rights reserved.*